In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf  #自相关图
from statsmodels.tsa.stattools import adfuller as ADF  #平稳性检测
from statsmodels.graphics.tsaplots import plot_pacf    #偏自相关图
from statsmodels.stats.diagnostic import acorr_ljungbox    #白噪声检验
from statsmodels.tsa.arima_model import ARIMA
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# 读取数据，选择4天长度作为训练集和测试集
metric_data=pd.read_csv("data/demo_data/metric/metric.csv")
metric_data=metric_data.loc[:287*4,:]

In [ ]:
# 切分训练集和测试集合
rate=0.75
series=metric_data["value"]
split_point = int(len(series)*rate)
dataset, validation = series[0:split_point], series[split_point:]
print('Dataset %d, Validation %d' % (len(dataset), len(validation)))
dataset.to_csv('dataset.csv', index=False)
validation.to_csv('validation.csv', index=False)

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
     return yhat + history[-interval]

In [ ]:
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
import numpy
 
# load dataset
series = read_csv('dataset.csv', header=0)
# seasonal difference
X = series.values
days_in_year = 365
differenced = difference(X, days_in_year)
# fit model
model = ARIMA(differenced, order=(1,2,1))
model_fit = model.fit()


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
freq=288
decomposition = seasonal_decompose(metric_data["value"], freq=freq, two_sided=False)
# self.ts:时间序列，series类型; 
# freq:周期，这里为1440分钟，即一天; 
# two_sided:观察下图2、4行图，左边空了一段，如果设为True，则会出现左右两边都空出来的情况，False保证序列在最后的时间也有数据，方便预测。

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
plt.show() 

In [ ]:
trend.dropna(inplace=True)
train = trend[:len(trend)-1088]
order=(1,2,1)
trend_model = ARIMA(train, order).fit(disp=-1, method='css')
# multi-step out-of-sample forecast
start_index = len(differenced)
end_index = start_index + 1088
forecast = model_fit.predict(start=start_index, end=end_index)
forecast